<a href="https://colab.research.google.com/github/zeinabmohamed/HistopathologicCancerDetection/blob/master/apply_fisher_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1 : Adding required imports : 

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 29 20:51:00 2019
@author: minamounir
"""
import numpy as np
import os
import imageio
import matplotlib.pylab as plt
import copy 
import pandas as pd
import sys
from numpy import genfromtxt

Step 2 : Amount the Google drive 

In [181]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Step 3  :  Read Data from CSV 

In [182]:
data_set_csv_path = "/content/drive/My Drive/histopathologic-cancer-detection/train_15000.csv"

import pandas as pd

df = pd.read_csv(data_set_csv_path, sep=',', header=None)
df.to_records(index=False)

#print("data set   " ,df)
print(df)

print(df.loc[0,0])
print(df.loc[0,1:None])



       0     1     2     3     4     5     ...  3067  3068  3069  3070  3071  3072
0         0    99    51   109   123    76  ...   175   151   165    96    76    88
1         0   130    56   155   161    79  ...   143    78   168   157    79   178
2         0   201   140   199   155    95  ...   248   246   247   249   247   248
3         1   228   196   209   255   232  ...   213   195   185   231   213   201
4         1    69    62   132    62    49  ...    82    46   133   179   141   226
...     ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
14995     0    88    21    64    73    11  ...    77    38    69   255   231   255
14996     0   215   171   188   227   183  ...   246   248   245   245   247   244
14997     0   218   212   226   215   207  ...   115    70   151   132    94   179
14998     0   112    79   110   196   166  ...   148    91   132   164   105   151
14999     1   193   139   171   131    83  ...   121    68    94   239   181   203

[15

Step 4 : Split Data set 80% train 20% test

In [183]:
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_test = df[~msk]
print("train data " ,df_train.shape)
print("test data " , df_test.shape)

train data  (12066, 3073)
test data  (2934, 3073)


In [0]:
def train(L,dataSet,C1,C2):
    print("train for Label",L)
    print("train data",dataSet.shape)

    for i in range(len(dataSet)):
        if(dataSet.iloc[i,0] == L):
            C1.append(dataSet.iloc[i,1:None])
        else:
            C2.append(dataSet.iloc[i,1:None])
    C1= np.asarray(C1)
    C2= np.asarray(C2)
    print(C1.shape)
    print(C2.shape)

In [0]:
    def getMean(DataSet): 
        return np.mean(DataSet, axis=0)

In [0]:

def getS1(C1,m1):
    # Calculate First Term of SW
    sumS1= np.zeros((3072,3072))
    for i in range(len(C1)):
        val =  np.subtract(C1[i], m1)
        #print("val len: ",len(val))
        sumS1 = sumS1 + np.dot(val.T,val) # MultiplyMatrix(val,val,0) 
    return sumS1


def getS2(C2,m2):
    # Calculate First Term of SW
    sumS2= np.zeros((3072,3072))
    for i in range(len(C2)):
        val =  np.subtract(C2[i], m2)
        #print("val len: ",len(val))
        sumS2 = sumS2 + np.dot(val.T,val) #MultiplyMatrix(val,val,0) 
    return sumS2

In [0]:
def MultiplyMatrix(a1,a2,tr):
    if tr==0:
        M1 = np.matrix(a1).T
        M2 = np.matrix(a2)
    else:
        M1 = np.matrix(a1)
        M2 = np.matrix(a2).T    
    return M1*M2

In [0]:
 def getSw(S1,S2):
    return np.add(S1,S2)

def get_W(Sw, mean1, mean2):
    m= 10^-6
    return np.dot(np.linalg.inv(Sw+np.eye(Sw.shape[1])*m), (mean2-mean1))

In [0]:
def get_Wo(w, m1, m2):
    negative_WT= np.negative(w)
    print("negative_WT : " , len(negative_WT))
    add_means = np.add(m1,m2)
    print("add_means : " , len(add_means))
    return np.dot(negative_WT.T,np.divide(add_means, 2)) #MultiplyMatrix(negative_WT, np.true_divide(add_means, 2),1)


In [0]:
W=np.zeros((2,3072))
W_o=np.zeros((2,1))

for Label in range (2):
    print("########## Loop",Label,"#############")
    C1=[]
    C2=[]
    train(Label,df_train,C1,C2)
    
    # get mean of class 1
    m1 = getMean(C1)

    # get mean of class 2
    m2 = getMean(C2)

    print(len(m1))
    print(len(m2))

    S1 = getS1(C1,m1)
    S2 = getS2(C2,m2)
    #print(S1(C1,m1))
    print("=========================================")
    #print(S2(C2,m2))
    Sw = getSw(S1,S2)
    w = get_W(Sw, m1, m2)
    W[Label]= copy.deepcopy(w)
    #print("SW : ",Sw)

    #print("Weight : ",w)
    w_o = get_Wo(w, m1, m2)
    W_o[Label]= copy.deepcopy(w_o)
    print("Wo : ",w_o)



In [0]:
Y_all = np.zeros((len(df_test),2))
Y_weights = np.zeros((2,1))


for i in range(len(df_test)):
    for j in range(2):
        w_traspose = W[j].T #1*3072
        test_row = df_test.iloc[i,1:] #3072
        Y=np.dot(w_traspose,test_row)+W_o[j]
        Y_weights[j]=Y
    Y_all[i,0]=df_test.iloc[i,0]
    Y_all[i,1]=  np.argmax(Y_weights)
    
print(Y_all)
        
#Calculate Accuracy and Print it
accuracy = 0

for i in range(len(Y_all)):
    if (Y_all[i,0] == Y_all[i,1]):
        accuracy = accuracy + 1
 
print("Accuracy = ",((accuracy/len(df_test))*100)," %")